<a href="https://colab.research.google.com/github/7ft10/JiraExporter/blob/main/DataFlows - Cloud/Jira_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#! Library Import

import os 
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

if importlib.util.find_spec("dotenv") is None:	
	os.system("pip install --quiet openai python-dotenv")

In [2]:
#! pandas Config

import pandas as pd
import requests
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

pd.options.mode.use_inf_as_na = True
#pd.set_option("display.max_rows", 10)
#pd.set_option("display.expand_frame_repr", True)
#pd.set_option('display.width', 1000)
#pd.options.display.max_seq_items = 200000
#pd.options.display.max_rows = 10


In [3]:
#! Parameters

import os 
import dotenv
import importlib.util

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

try:
    dotenv.load_dotenv('.env')

    Host = os.getenv('SECRETS_HOST') 
    Username = os.getenv('SECRETS_USERNAME') 
    Password = os.getenv('SECRETS_PASSWORD') 
except: 
    pass 

if Host is None or Host == "":
    Host = input("Enter Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Host: " + Host)

ValidProjectCategories = ["'Customer Delivery Projects'"]
display("ValidProjectCategories: " + ','.join(ValidProjectCategories))

'Host: https://strategenics.atlassian.net/'

"ValidProjectCategories: 'Customer Delivery Projects'"

In [4]:
#! Functions

import time
import base64
import pandas as pd
import re 
import requests
import warnings
from functools import reduce
    
def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    elif (prefix != "" and prefix is not None):
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))
    
    if len(columnsToExpand) > 0:        
        expandedCols = expandedCols[columnsToExpand]
    
    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns] 

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0" 
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame: 
    self.columns = [fnSentenceCase(c) for c in self.columns] 
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        
        response = requests.post(Host + url, headers = headers, json = defaultContents, verify=False)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes) -> pd.DataFrame:
    def ApiCall(_) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values).strip(),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "").strip()
        }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = df.expand("projects", [], None, False)    
    try:
        df = df[["issuetypes"]]
    except: 
        raise Exception("No issue metadata - check the valid project categories are correct")
    df = df.explode("issuetypes")
    df = df.expand("issuetypes", [], None, False)
    df = df[["fields"]]
    df = df.expand("fields", [], None, False)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid:pd.DataFrame = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'] if "key" in valid else valid["fieldId"],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            #display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'id', "name": 'Id', "schema_type": 'number', "required": True })
    values.append({ "fieldId": 'key', "name": 'Key', "schema_type": 'string', "required": True })
    df = pd.DataFrame(values)
    df = df.drop_duplicates().sort_values("fieldId")
    return df 

def fnGetValidProjectKeys() -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "/rest/api/latest/project/search"
        headers = fnGetDefaultHeaders()
        params = { 
            "startAt": startAt,
            "maxResults": 100
        }
        response = requests.get(Host + url, headers = headers, params = params, verify = False)
        return response.json()
    df = fnAPI(ApiCall)    
    df = df.explode("values")
    df = df[["values"]]
    df = df.expand("values", ["key", "projectCategory"], None, False)
    df = df.expand("projectCategory")
    if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0:
        df = df.loc[df['Project Category Name'].isin([ c.replace("'", "") for c in ValidProjectCategories ])]
    return df[["key"]]

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

def fnGetTimeZoneOffset() -> str:
    offset = (time.timezone if (time.localtime().tm_isdst == 0) else time.altzone) / 60 / 60 * -1
    return ("" if(offset) < 0 else "+") + str(int((offset - (offset % 1)))).zfill(2) + ":" + str(int((offset % 1) * 60)).zfill(2)

goldenDF = None
globals()['goldenDF'] = None 

def fnGetGoldenCopy() -> pd.DataFrame:
    if 'goldenDF' not in globals() or goldenDF is None: 
        raise Exception("Base data frame not loaded") 
    else:
        df:pd.DataFrame = goldenDF.copy(deep = True)
        if df is None or len(df.index) == 0:
            raise Exception ("No results")
    return df

def exit():
    class StopExecution(Exception):
        def _render_traceback_(self):
            []
    raise StopExecution


In [5]:
#! Jira Issues Capture

from IPython.display import display

ExpectedIssueTypes = [ "Task" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") ORDER BY updatedDate DESC"

fields = fnGetIssueTypeFields(ExpectedIssueTypes)

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
	exit()

df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
df = df.explode("issues")
df = df.expand("issues", [], None, False)
df = df.drop(["expand", "self"], axis=1)
df = df.expand("fields", [], None, False)		

df = df.rename( columns=dict( zip ( fields.fieldId, fields.name )) )	

df = df.convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')

goldenDF:pd.DataFrame = df.copy(deep = True)
globals()['goldenDF'] = goldenDF # make this globally available 

df:pd.DataFrame = fnGetGoldenCopy()
display(df.dtypes)
display(df.head())

Id                          Int64
Key                string[python]
Issue Type                 object
Components                 object
Description        string[python]
Project                    object
Fix versions               object
Resolution                 object
Tempo Customer             object
Sprint                     object
Epic Link          string[python]
Attachment                 object
Resolution Date    string[python]
Summary            string[python]
Last Viewed        string[python]
Created            string[python]
Reporter                   object
Priority                   object
Story Points               object
Labels                     object
Account                    object
Start date         string[python]
Due date           string[python]
Linked Issues              object
Assignee                   object
Updated            string[python]
Status                     object
Parent                     object
dtype: object

,Id,Key,Issue Type,Components,Description,Project,Fix versions,Resolution,Tempo Customer,Sprint,...,Story Points,Labels,Account,Start date,Due date,Linked Issues,Assignee,Updated,Status,Parent
0,50817,SAT-116,{'self': 'https://strategenics.atlassian.net/r...,[{'self': 'https://strategenics.atlassian.net/...,<NA>,{'self': 'https://strategenics.atlassian.net/r...,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,...,None,[],"{'id': 37, 'value': 'Auto & General - T&M'}",2023-07-21,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,2023-08-02T09:18:42.800+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN
1,50991,SAT-134,{'self': 'https://strategenics.atlassian.net/r...,[{'self': 'https://strategenics.atlassian.net/...,"* Jez request … ""Get our consulting packages (...",{'self': 'https://strategenics.atlassian.net/r...,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,...,None,[],None,2023-08-01,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,2023-08-01T18:12:02.822+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN
2,50731,SAT-60,{'self': 'https://strategenics.atlassian.net/r...,[{'self': 'https://strategenics.atlassian.net/...,"Policy, process and procedures - find, update ...",{'self': 'https://strategenics.atlassian.net/r...,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,...,None,[],None,2023-07-21,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,2023-08-01T14:21:49.188+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN
3,50977,SAT-133,{'self': 'https://strategenics.atlassian.net/r...,[],Review the report from the CyberGRX audit and ...,{'self': 'https://strategenics.atlassian.net/r...,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,...,None,[],None,2023-08-01,<NA>,[],{'self': 'https://strategenics.atlassian.net/r...,2023-08-01T14:21:33.863+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN
4,50879,SAT-118,{'self': 'https://strategenics.atlassian.net/r...,[],AI white paper has been developed for a prior ...,{'self': 'https://strategenics.atlassian.net/r...,[],None,{'self': 'https://strategenics.atlassian.net/r...,None,...,None,[],None,2023-07-25,<NA>,"[{'id': '36107', 'self': 'https://strategenics...",{'self': 'https://strategenics.atlassian.net/r...,2023-07-31T18:32:53.570+1000,{'self': 'https://strategenics.atlassian.net/r...,NaN


In [6]:
#! Issues

from IPython.display import display

df:pd.DataFrame = fnGetGoldenCopy()
df = df.drop(["Components", "Attachment", "Linked Issues", "Sprint", "Fix versions", "Labels"], axis=1)
df = df.expand("Issue Type", ["id", "name"])
df = df.expand("Status", ["id", "name", "statusCategory"]).sentence_case_columns()
df = df.expand("Status Category", ["id", "name"]) 
df = df.expand("Project", ["id", "name", "projectCategory"]).sentence_case_columns()
df = df.expand("Project Category", ["id", "name"])
df = df.expand("Reporter", ["accountId", "displayName"])
df = df.expand("Assignee", ["accountId", "displayName"])
df = df.expand("Priority", ["id", "name"])
df = df.expand("Resolution", ["id", "name"])
df = df.expand("Parent", ["id", "key"])     
df = df.expand("Account", ["id", "value"])   
df = df.expand("Tempo Customer", ["id", "value"])   

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Story Points"] = df["Story Points"].astype('float')

display(df.dtypes)
display(df.head())

Id                                Int64
Key                      string[python]
Description              string[python]
Epic Link                string[python]
Resolution Date          string[python]
Summary                  string[python]
Last Viewed              string[python]
Created                  string[python]
Story Points                    float64
Start Date               string[python]
Due Date                 string[python]
Updated                  string[python]
Issue Type Id            string[python]
Issue Type Name          string[python]
Status Id                string[python]
Status Name              string[python]
Status Category Id                Int64
Status Category Name     string[python]
Project Id               string[python]
Project Name             string[python]
Project Category Id      string[python]
Project Category Name    string[python]
Reporter Account Id      string[python]
Reporter Display Name    string[python]
Assignee Account Id      string[python]


,Id,Key,Description,Epic Link,Resolution Date,Summary,Last Viewed,Created,Story Points,Start Date,...,Priority Id,Priority Name,Resolution Id,Resolution Name,Parent Id,Parent Key,Account Id,Account Value,Tempo Customer Id,Tempo Customer Value
0,50817,SAT-116,<NA>,<NA>,<NA>,Auto & General Reporting,2023-07-21T17:31:12.763+1000,2023-07-21T16:27:38.239+1000,NaN,2023-07-21,...,3,Medium,<NA>,<NA>,<NA>,<NA>,37,Auto & General - T&M,10578,Auto & General
1,50991,SAT-134,"* Jez request … ""Get our consulting packages (...",<NA>,<NA>,AWS Marketplace consulting packages (fixed price),2023-08-02T08:47:22.065+1000,2023-08-01T18:11:52.879+1000,NaN,2023-08-01,...,3,Medium,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,10556,No Customer Selected
2,50731,SAT-60,"Policy, process and procedures - find, update ...",<NA>,<NA>,Strategenics - Cybersecurity Policy,2023-07-20T14:28:31.629+1000,2023-07-20T11:26:46.616+1000,NaN,2023-07-21,...,3,Medium,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,10558,Strategenics
3,50977,SAT-133,Review the report from the CyberGRX audit and ...,<NA>,<NA>,Analyise the CyberGRX audit report,<NA>,2023-08-01T10:39:19.148+1000,NaN,2023-08-01,...,3,Medium,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,10556,No Customer Selected
4,50879,SAT-118,AI white paper has been developed for a prior ...,<NA>,<NA>,Refresh AI white paper for Gartner,<NA>,2023-07-25T07:53:45.841+1000,NaN,2023-07-25,...,3,Medium,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,10556,No Customer Selected


In [7]:
#! Components

from IPython.display import display

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Components"]]
df = df[df["Components"].map(lambda d: len(d)) > 0]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Components")
df = df.expand("Components", ["id", "name"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

display(df.dtypes)
display(df.head())

Id                          Int64
Key                string[python]
Components Id      string[python]
Components Name    string[python]
dtype: object

,Id,Key,Components Id,Components Name
0,50817,SAT-116,12452,Consulting
1,50991,SAT-134,12446,AWS
2,50991,SAT-134,12452,Consulting
3,50991,SAT-134,12439,Marketing
4,50991,SAT-134,12438,Product Development


In [8]:
#! Linked Issues

from IPython.display import display

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Linked Issues"]]
df = df[df["Linked Issues"].map(lambda d: len(d)) > 0]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Linked Issues")
df = df.expand("Linked Issues", ["id", "type", "inwardIssue", "outwardIssue"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

display(df.dtypes)
display(df.head())

Id                                      Int64
Key                            string[python]
Linked Issues Id               string[python]
Linked Issues Type                     object
Linked Issues Inward Issue             object
Linked Issues Outward Issue            object
dtype: object

,Id,Key,Linked Issues Id,Linked Issues Type,Linked Issues Inward Issue,Linked Issues Outward Issue
0,50879,SAT-118,36107,"{'id': '10000', 'name': 'Blocks', 'inward': 'i...",NaN,"{'id': '50025', 'key': 'MKTG-616', 'self': 'ht..."
1,50879,SAT-118,36108,"{'id': '10000', 'name': 'Blocks', 'inward': 'i...","{'id': '50054', 'key': 'MKTG-627', 'self': 'ht...",NaN
2,50879,SAT-118,36105,"{'id': '10000', 'name': 'Blocks', 'inward': 'i...","{'id': '50133', 'key': 'MKTG-644', 'self': 'ht...",NaN
3,50879,SAT-118,36106,"{'id': '10407', 'name': 'Parent-Child', 'inwar...","{'id': '49995', 'key': 'MKTG-596', 'self': 'ht...",NaN
4,50935,SAT-129,36085,"{'id': '10407', 'name': 'Parent-Child', 'inwar...","{'id': '50763', 'key': 'SAT-92', 'self': 'http...",NaN


In [9]:
#! Sprints

from IPython.display import display
from pandas.core.dtypes.dtypes import DatetimeTZDtype

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Sprint"]]
df = df[~df["Sprint"].isna()]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Sprint")
df = df.expand("Sprint")

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

df["Sprint Start Date"] = df["Sprint Start Date"].astype(DatetimeTZDtype("ns", fnGetTimeZoneOffset()))
df["Sprint End Date"] = df["Sprint End Date"].astype(DatetimeTZDtype("ns", fnGetTimeZoneOffset()))
df["Sprint Complete Date"] = df["Sprint Complete Date"].astype(DatetimeTZDtype("ns", fnGetTimeZoneOffset()))

display(df.dtypes)
display(df.head())

'No results'

In [10]:
#! Attachments

from IPython.display import display
from pandas.core.dtypes.dtypes import DatetimeTZDtype

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Attachment"]]
df = df[df["Attachment"].map(lambda d: len(d)) > 0]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Attachment")
df = df.expand("Attachment", ["id", "filename", "created", "mimeType", "size", "thumbnail", "content", "author"])
df = df.expand("Attachment Author", ["accountId"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Attachment Id"] = df["Attachment Id"].astype('Int64')
df["Attachment Size"] = df["Attachment Size"].astype('float64')
df["Attachment Created"] = df["Attachment Created"].astype(DatetimeTZDtype("ns", fnGetTimeZoneOffset()))

display(df.dtypes)
display(df.head())

Id                                                  Int64
Key                                        string[python]
Attachment Id                                       Int64
Attachment Filename                        string[python]
Attachment Created              datetime64[ns, UTC+10:00]
Attachment Mime Type                       string[python]
Attachment Size                                   float64
Attachment Thumbnail                       string[python]
Attachment Content                         string[python]
Attachment Author Account Id               string[python]
dtype: object

,Id,Key,Attachment Id,Attachment Filename,Attachment Created,Attachment Mime Type,Attachment Size,Attachment Thumbnail,Attachment Content,Attachment Author Account Id
0,50977,SAT-133,38176,20230801_Strategenics_T2_Mapped_INDP_Australia...,2023-08-01 14:20:00.593000+10:00,application/vnd.openxmlformats-officedocument....,56001.0,<NA>,https://strategenics.atlassian.net/rest/api/2/...,712020:dee1500a-4549-41d1-a24f-e825c60a436e
1,50977,SAT-133,38183,20230801_Strategenics_T2_Mapped_INDP_Australia...,2023-08-01 14:20:01.083000+10:00,application/vnd.openxmlformats-officedocument....,16363.0,<NA>,https://strategenics.atlassian.net/rest/api/2/...,712020:dee1500a-4549-41d1-a24f-e825c60a436e
2,50977,SAT-133,38172,20230801_Strategenics_T2_Mapped_STND_ACSC Esse...,2023-08-01 14:20:00.362000+10:00,application/vnd.openxmlformats-officedocument....,19639.0,<NA>,https://strategenics.atlassian.net/rest/api/2/...,712020:dee1500a-4549-41d1-a24f-e825c60a436e
3,50977,SAT-133,38179,20230801_Strategenics_T2_Mapped_STND_AUS Signa...,2023-08-01 14:20:00.707000+10:00,application/vnd.openxmlformats-officedocument....,20671.0,<NA>,https://strategenics.atlassian.net/rest/api/2/...,712020:dee1500a-4549-41d1-a24f-e825c60a436e
4,50977,SAT-133,38177,20230801_Strategenics_T2_Mapped_STND_CMMC-Lvl ...,2023-08-01 14:20:00.586000+10:00,application/vnd.openxmlformats-officedocument....,13874.0,<NA>,https://strategenics.atlassian.net/rest/api/2/...,712020:dee1500a-4549-41d1-a24f-e825c60a436e


In [11]:
#! Versions

from IPython.display import display

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Fix versions"]]
df = df[df["Fix versions"].map(lambda d: len(d)) > 0]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Fix versions")
df = df.expand("Fix versions", ["id", "name"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Fix Versions Id"] = df["Fix Versions Id"].astype('Int64')

display(df.dtypes)
display(df.head())

Id                            Int64
Key                  string[python]
Fix Versions Id               Int64
Fix Versions Name    string[python]
dtype: object

,Id,Key,Fix Versions Id,Fix Versions Name
0,50747,SAT-76,11581,Gartner IT Symposium/Xpo™ 2023 Australia
1,50732,SAT-61,11581,Gartner IT Symposium/Xpo™ 2023 Australia
2,50749,SAT-78,11581,Gartner IT Symposium/Xpo™ 2023 Australia
3,50748,SAT-77,11581,Gartner IT Symposium/Xpo™ 2023 Australia
4,44399,TRC-325,11540,PMIS Procurement Package
